In [2]:
from PIL import Image
from rl_benchmarks.models import iBOTViT
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from rl_benchmarks.utils.linear_evaluation import get_binary_class_metrics, get_bootstrapped_metrics

from PIL import Image
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
# import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, cohen_kappa_score, accuracy_score
import torch.nn.functional as F
import sys
import time

import shutil
import os

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import cv2
import time
from sklearn.preprocessing import label_binarize

import timm
from metrics import report
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset, Dataset
from tqdm import tqdm
from rl_benchmarks.metrics import *

from rl_benchmarks.trainers.torch_trainer import TorchTrainer
from rl_benchmarks.models.slide_models.meanpool import MeanPool
from rl_benchmarks.models.slide_models.chowder import Chowder
from rl_benchmarks.models.slide_models.dsmil import DSMIL
from rl_benchmarks.models.slide_models.abmil import ABMIL
from rl_benchmarks.models.slide_models.hiptmil import HIPTMIL
from rl_benchmarks.models.slide_models.transmil import TransMIL

from pathlib import Path
from metrics import report

import sys
sys.path.append('/home/yuxin/bme/BCaCAD/model')
from patch_based_test.img import QiLuROI

def get_model(model_name, in_dim, out_dim):
    if model_name == 'abmil':
        model = ABMIL(in_dim, out_dim, d_model_attention=128,temperature= 1.0, mlp_hidden= [128, 64])
    elif model_name == 'chowder':
        model = Chowder(in_dim, out_dim, n_top= 2,n_bottom= 2,tiles_mlp_hidden= [128], mlp_hidden= [128, 64])
    elif model_name == 'dsmil':
        model = DSMIL(in_dim, out_dim, d_tiles_values= 32,d_tiles_queries= 32,passing_values= False,tiles_scores_mlp_hidden=[200,100],
                        tiles_queries_mlp_hidden=[200,100], mlp_hidden=[200,100])
    elif model_name == 'hiptmil':
        model = HIPTMIL(in_dim, out_dim)
    elif model_name == 'transmil':
        model = TransMIL(in_dim, out_features=out_dim)
    elif model_name == 'meanpool':
        model = MeanPool(in_dim, out_dim)
    else:
        raise 'model not found'
    return model


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
num_classes = [3,3]
img_size = patch_size = 384
data_trans = {
    "train": transforms.Compose([
                                transforms.Resize(img_size),
                                transforms.ColorJitter(),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    "test": transforms.Compose([
                                transforms.Resize((img_size,img_size)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

In [4]:
class Feature(Dataset):
    def get_label(self, type):
        D = {
            'normal':[0,0],
            'dcis-1':[1,0],
            'dcis-2':[1,1],
            'dcis-3':[1,2],
            'ibc-1':[2,0],
            'ibc-2':[2,1],
            'ibc-3':[2,2],
        }
        label =  D[type]
        return label[0],label[1]
    
    def __init__(self, feature_dir, phase='test'):
        folder = Path(feature_dir)
        feature_paths = list(folder.rglob('**/*.npy'))
        self.labels = [self.get_label(path.parent.parent.name) for path in feature_paths]
        self.features = [np.load(path) for path in feature_paths]
        self.phase=phase
        
    def __len__(self):
        return len(self.features)
    def __getitem__(self, item):
        features = self.features[item]
        if self.phase == 'train':
            indices = np.random.choice(features.shape[0], 8, replace=True)
            features = np.stack([features[i] for i in indices], axis=0)
        return features, self.labels[item]
    
    @staticmethod
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images,  labels = tuple(zip(*batch))

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        # masks = torch.as_tensor(masks)

        return images, labels


In [5]:
# data_root = Path('/mnt/hd0/project/bcacad/data/roi-level')
feature_root = Path('/mnt/hd0/project/bcacad/model/wsi_features')
test_cohorts = ['suqh_all_patch', 'qduh', 'shsu', 'bracs', 'bcnb', 'bach', 'apght']
# test_cohorts = ['suqh_all_patch']


In [6]:
criterion = nn.CrossEntropyLoss()
merics = {'acc': compute_multiclass_accuracy, 'auc': compute_mean_one_vs_all_auc}
input_dim = 768
out_dim = [3,3]
tasks = ['type', 'nonibc', 'ibc']
class_names = {
    'type': ['Normal', 'nonIBC', 'IBC'],
    'nonibc': ['Low', 'Medium', 'High'],
    'ibc': ['Low', 'Medium', 'High'],
}

In [7]:
model_names = [ 'abmil']
models={}
model_dir = Path('/mnt/hd0/project/bcacad/model/roi_models/model6')
for model_name in model_names:
    model = get_model(model_name, input_dim, out_dim).to(device)
    model.load_state_dict(torch.load(model_dir / f'{model_name}.pth'))
    models[model_name] = model.to(device)

In [13]:
train_set = Feature(feature_root / 'suqh' / 'model', 'train')

In [ ]:
X = []
Y = []
for i in range(len(train_set)):
    x, y = train_set[i]
    x = torch.Tensor(x).to(device)
    x = x.unsqueeze(0)
    # print(x.shape)
    output = models['abmil'](x)
    output = [output[0].cpu().detach().numpy(), output[1].cpu().detach().numpy()]
    output = np.array(output).reshape(-1)
    X.append(output)
    Y.append(y)
X = np.array(X)
Y = np.array(Y)

In [29]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

clf = sklearn.linear_model.SGDClassifier(
                loss="log_loss",
                penalty="l2",
                learning_rate="adaptive",
                eta0=1e-4,
                n_jobs=8,
                early_stopping=False,
                random_state=2023,
            )
mclf = MultiOutputClassifier(clf, n_jobs=10)

In [31]:
mclf.fit(X, Y)

MultiOutputClassifier(estimator=SGDClassifier(eta0=0.0001,
                                              learning_rate='adaptive',
                                              loss='log_loss', n_jobs=8,
                                              random_state=2023),
                      n_jobs=10)

In [10]:
def get_max_index(preds):
    types = preds[0]
    grades = preds[1]
    max_grade = -1
    max_type = -1
    max_index = -1

    for i in range(len(preds[0])):
        t = types[i]
        g = grades[i]
        if t > max_type or (t == max_type and g > max_grade):
            max_type = t
            max_grade = g
            max_index = i
    return max_index
    

In [11]:
def get_majority_index(preds):
    types = preds[0]
    grades = preds[1]
    type_counts = [0,0,0]
    grade_counts = [0,0,0]
    for i in range(len(types)):
        type_counts[types[i]] += 1
        grade_counts[grades[i]] += 1
    max_type = type_counts.index(max(type_counts))
    max_grade = grade_counts.index(max(grade_counts))
    for i in range(len(types)):
        if types[i] == max_type and grades[i] == max_grade:
            return i
    return -1

In [16]:
# testing
pha='test'
# test_cohorts = ['suqh_all_patch', 'qduh', 'shsu', 'bracs', 'bcnb', 'bach', 'apght']
test_cohorts = [ 'suqh_all_patch']
# load_dir = Path('/mnt/hd0/project/bcacad/model/roi_models/model2')
for cohort in test_cohorts:
    for model_name in model_names:
        # model = get_model(model_name, input_dim, out_dim).to(device)
        model = models[model_name]
        # ckpt = torch.load(load_dir / f'{model_name}.pth')
        # model.load_state_dict(ckpt)
        # model = model.to(device)
        wsi_dirs = list((feature_root / cohort / 'test').glob('*/*'))
        test_sets = [Feature(wsi_dir, 'test') for wsi_dir in wsi_dirs]
        # test_set = FeatureWSI(feature_root / cohort / 'test', 'test')
        # trainer = TorchTrainer(model, criterion, merics, device=device, num_epochs=50)
        Labels =[[],[]]
        Preds = [[],[]]
        Probs = [[],[]]
        for test_set in test_sets:
            res = trainer.predict(test_set)

            labels = np.array(res[0])
            probs = np.array(res[1])
            preds = np.array(res[2])
            max_idx = get_max_index(preds)
            # max_idx = get_majority_index(preds)
            label = labels[0][max_idx], labels[1][max_idx]
            prob = probs[0][max_idx], probs[1][max_idx]
            pred = preds[0][max_idx], preds[1][max_idx]
            Labels[0].append(label[0])
            Labels[1].append(label[1])
            Probs[0].append(prob[0])
            Probs[1].append(prob[1])
            Preds[0].append(pred[0])
            Preds[1].append(pred[1])

        
        labels = np.array(Labels)
        probs = np.array(Probs)
        preds = np.array(Preds)
            

        type_labels = labels[0]
        type_probs = probs[0]
        type_preds = preds[0]

        nonibc_index = np.where(type_labels ==1)
        nonibc_labels = labels[1][nonibc_index]
        nonibc_probs = probs[1][nonibc_index]
        nonibc_preds = preds[1][nonibc_index]

        ibc_index = np.where(type_labels ==2)
        ibc_labels = labels[1][ibc_index]
        ibc_probs = probs[1][ibc_index]
        ibc_preds = preds[1][ibc_index]

        re = {}
        avg_aucs = {}
        if type_labels.size == 0:
            re['type'] = None
            avg_aucs['type'] = None
        else:
            re['type'] = report(type_labels, type_preds, type_probs, class_names['type'])
            avg_aucs['type'] = compute_mean_one_vs_all_auc(type_labels, type_probs)

        if nonibc_labels.size == 0:
            re['nonibc'] = None
            avg_aucs['nonibc'] = None
        else:
            re['nonibc'] = report(nonibc_labels, nonibc_preds, nonibc_probs, class_names['nonibc'])
            avg_aucs['nonibc'] = compute_mean_one_vs_all_auc(nonibc_labels, nonibc_probs)

        if ibc_labels.size == 0:
            re['ibc'] = None
            avg_aucs['ibc'] = None
        else:
            re['ibc'] = report(ibc_labels, ibc_preds, ibc_probs, class_names['ibc'])
            avg_aucs['ibc'] = compute_mean_one_vs_all_auc(ibc_labels, ibc_probs)

        for task in ['type', 'nonibc', 'ibc']:
            r = re[task]
            if r is None:
                continue
            fs = "{} {} {} acc: {:.4f}, auc: {:.4f} [{:.4f} {:.4f} {:.4f}]".format(cohort, model_name, task, r['accuracy'], avg_aucs[task], r['0']['auc'], r['1']['auc'], r['2']['auc'])
            print(fs)
        print()

bcnb abmil type acc: 0.9698, auc: nan [nan nan nan]
bcnb abmil ibc acc: 0.5238, auc: 0.6634 [0.7236 0.5962 0.6707]



In [17]:
save_root = Path('/mnt/hd0/project/bcacad/model/roi_models/model7')
if not save_root.exists():
    save_root.mkdir()
for model_name in model_names:
    torch.save(models[model_name].state_dict(), save_root / f'{model_name}.pth')
    print(f'save {model_name}.pth')

save abmil.pth
save transmil.pth


In [29]:
test_set = FeatureWSI(feature_root / cohort / 'test', 'test')

In [34]:
len(test_set.features[3])

9

In [37]:
np.concatenate(test_set.features[3], axis=0).shape

(36, 768)